### New Model Solution

In [1]:
import math
import numpy
from gurobipy import *
import os

p = 0.8
c = 0
h_List = [182,660,1155,465,46,218,270,345,1292,2235,2770,1350,195,2129,8697,2775,172,27,900,17201,1279,21780]
b_List = [1000,1000,1000,1000,1000,1000,1000,1000,5000,1000,5000,1000,1000,5000,10000,5000,1000,1000,1000,20000,5000,40000]
L_List = [19,15,8,9,9,6,9,8,16,15,6,7,2,7,5,10,11,35,10,8,7,4]
F_S_S = []
solution = []

S_MAX = 51
B_MAX = 11
N = 22
Structure = [[-1],[-1],[-1],[-1],[-1],[-1],[-1],[-1],[0,1],[-1],[2,3,4],[-1],[-1],[5,6,7],[8,9,10],[11,12],[-1],[-1],[-1],[13,14,15],[16,17],[18,19,20]]
S_in_supplier = [10,10,10,10,10,10,10,10,1e20,10,1e20,10,10,1e20,1e20,1e20,10,10,10,1e20,1e20,1e20]
S_out_customer = [1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,5]

for i in range (N): #re-scaling
    b_List[i] = b_List[i] *100 / 100.0
    h_List[i] = h_List[i] / 100.0

FSSFile = open("FSSFile.txt", mode='w')

def Cost (j,B,W,S_out):
    if (B == 0):
        if (W+L_List[j]-S_out > 0):
            return c+b_List[j]
        else:
            return c
    sum_1 = 0
    for x in range (0,int(W+L_List[j]-S_out-29*B)):
        sum_1 += math.factorial(B)/math.factorial(x)/math.factorial(B-x)*(p**x)*((1-p)**(B-x))
    sum_2 = 0
    for k in range (1,int(30*B-W-L_List[j]+S_out+1)):
        x = W+k+L_List[j]-S_out-29*B
        if (x < 0):
            continue
        sum_2 += k*h_List[j]*math.factorial(B)/math.factorial(x)/math.factorial(B-x)*(p**x)*((1-p)**(B-x))
    cost = c+b_List[j]*sum_1+sum_2
    return cost

for S_in in range (0,S_MAX):
    temp = []
    for S_out in range (0,S_MAX):
        BaseStock = numpy.ones(N).tolist()
        Waiting = numpy.zeros(N).tolist()
        SubCost = []
        for j in range (N):
            SubCost.append(5000)
        for j in range (N):
            for B in range (0,B_MAX):
                for W in range (max(S_in,S_out+29*B-L_List[j]), int(30*B+S_out-L_List[j]+1)):
                    if (SubCost[j] > Cost(j,B,W,S_out)):
                        SubCost[j] = Cost(j,B,W,S_out)
                        BaseStock[j] = B
                        Waiting[j] = W
            FSSFile.write(str(j)+'\t'+str(S_in)+'\t'+str(S_out)+'\t'+str(SubCost[j])+'\t'+str(BaseStock[j])+'\t'+str(Waiting[j])+'\n')
        temp.append([SubCost,BaseStock,Waiting])
        print(S_in,S_out,"success")
    F_S_S.append(temp)

F_S_S = numpy.array(F_S_S)
print("F_S_S generated!")
FSSFile.close()

'''
FSSFile = open("FSSFile.txt", mode='r')
FSS_temp = []
while True:
    FSSLine = FSSFile.readline()
    if FSSLine:
        FSSLine = FSSLine[:-1]
        FSS_temp.append(list(map(float,FSSLine.split('\t'))))
    else:
        break
for S_in in range (0,S_MAX):
    temp = []
    for S_out in range (0,S_MAX):
        BaseStock = numpy.ones(N).tolist()
        Waiting = numpy.zeros(N).tolist()
        SubCost = []
        for j in range (N):
            SubCost.append(1e20)
        for j in range (N):
            SubCost[j] = FSS_temp[N*S_MAX*S_in+N*S_out+j][3]
            BaseStock[j] = int(FSS_temp[N*S_MAX*S_in+N*S_out+j][4])
            Waiting[j] = int(FSS_temp[N*S_MAX*S_in+N*S_out+j][5])
        temp.append([SubCost,BaseStock,Waiting])
    F_S_S.append(temp)
F_S_S = numpy.array(F_S_S)
'''

Non_Start = [] #find the set of stages with upstream stages
for j in range (N):
    if (Structure[j][0] > -1):
        Non_Start.append(j)
    
#Solve MILP
try:
    m = Model("Algo-New")
    # Create variables
    chi = m.addVars(N, S_MAX, S_MAX, vtype=GRB.BINARY, name="chi")
    # Set objective
    obj = 0
    for j in range (N):
        for mu in range (S_MAX):
            for nu in range (S_MAX):
                obj += F_S_S[mu,nu,0,j]*chi[j,mu,nu]
    m.setObjective(obj, GRB.MINIMIZE)
    # Add constraint
    for j in range (N):
        m.addConstr(quicksum(chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == 1)
        if (S_out_customer[j] < 1e20):
            m.addConstr(quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_out_customer[j])
        if (S_in_supplier[j] < 1e20):
            m.addConstr(quicksum(mu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_in_supplier[j])
    for i in range (len(Non_Start)):
        for j in Structure[Non_Start[i]]:
            m.addConstr(quicksum(mu*chi[Non_Start[i],mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) >= quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)))
    
    m.optimize()

except GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')
    
for j in range (N):
    for mu in range (S_MAX):
        for nu in range (S_MAX):
            if(chi[j,mu,nu].X == 1):
                solution.append([F_S_S[mu,nu,0,j],mu,nu,F_S_S[mu,nu,1,j],F_S_S[mu,nu,2,j]]) #obj,S_in,S_out,B,W
solution = numpy.array(solution)
print("solution:",solution)
total_cost = 0
for j in range (N):
    total_cost += solution[j,0]
print("optimal total cost:", total_cost)

0 0 success
0 1 success
0 2 success
0 3 success
0 4 success
0 5 success
0 6 success
0 7 success
0 8 success
0 9 success
0 10 success
0 11 success
0 12 success
0 13 success
0 14 success
0 15 success
0 16 success
0 17 success
0 18 success
0 19 success
0 20 success
0 21 success
0 22 success
0 23 success
0 24 success
0 25 success
0 26 success
0 27 success
0 28 success
0 29 success
0 30 success
0 31 success
0 32 success
0 33 success
0 34 success
0 35 success
0 36 success
0 37 success
0 38 success
0 39 success
0 40 success
0 41 success
0 42 success
0 43 success
0 44 success
0 45 success
0 46 success
0 47 success
0 48 success
0 49 success
0 50 success
1 0 success
1 1 success
1 2 success
1 3 success
1 4 success
1 5 success
1 6 success
1 7 success
1 8 success
1 9 success
1 10 success
1 11 success
1 12 success
1 13 success
1 14 success
1 15 success
1 16 success
1 17 success
1 18 success
1 19 success
1 20 success
1 21 success
1 22 success
1 23 success
1 24 success
1 25 success
1 26 success
1 27 s

12 28 success
12 29 success
12 30 success
12 31 success
12 32 success
12 33 success
12 34 success
12 35 success
12 36 success
12 37 success
12 38 success
12 39 success
12 40 success
12 41 success
12 42 success
12 43 success
12 44 success
12 45 success
12 46 success
12 47 success
12 48 success
12 49 success
12 50 success
13 0 success
13 1 success
13 2 success
13 3 success
13 4 success
13 5 success
13 6 success
13 7 success
13 8 success
13 9 success
13 10 success
13 11 success
13 12 success
13 13 success
13 14 success
13 15 success
13 16 success
13 17 success
13 18 success
13 19 success
13 20 success
13 21 success
13 22 success
13 23 success
13 24 success
13 25 success
13 26 success
13 27 success
13 28 success
13 29 success
13 30 success
13 31 success
13 32 success
13 33 success
13 34 success
13 35 success
13 36 success
13 37 success
13 38 success
13 39 success
13 40 success
13 41 success
13 42 success
13 43 success
13 44 success
13 45 success
13 46 success
13 47 success
13 48 success
13

24 27 success
24 28 success
24 29 success
24 30 success
24 31 success
24 32 success
24 33 success
24 34 success
24 35 success
24 36 success
24 37 success
24 38 success
24 39 success
24 40 success
24 41 success
24 42 success
24 43 success
24 44 success
24 45 success
24 46 success
24 47 success
24 48 success
24 49 success
24 50 success
25 0 success
25 1 success
25 2 success
25 3 success
25 4 success
25 5 success
25 6 success
25 7 success
25 8 success
25 9 success
25 10 success
25 11 success
25 12 success
25 13 success
25 14 success
25 15 success
25 16 success
25 17 success
25 18 success
25 19 success
25 20 success
25 21 success
25 22 success
25 23 success
25 24 success
25 25 success
25 26 success
25 27 success
25 28 success
25 29 success
25 30 success
25 31 success
25 32 success
25 33 success
25 34 success
25 35 success
25 36 success
25 37 success
25 38 success
25 39 success
25 40 success
25 41 success
25 42 success
25 43 success
25 44 success
25 45 success
25 46 success
25 47 success
25

36 16 success
36 17 success
36 18 success
36 19 success
36 20 success
36 21 success
36 22 success
36 23 success
36 24 success
36 25 success
36 26 success
36 27 success
36 28 success
36 29 success
36 30 success
36 31 success
36 32 success
36 33 success
36 34 success
36 35 success
36 36 success
36 37 success
36 38 success
36 39 success
36 40 success
36 41 success
36 42 success
36 43 success
36 44 success
36 45 success
36 46 success
36 47 success
36 48 success
36 49 success
36 50 success
37 0 success
37 1 success
37 2 success
37 3 success
37 4 success
37 5 success
37 6 success
37 7 success
37 8 success
37 9 success
37 10 success
37 11 success
37 12 success
37 13 success
37 14 success
37 15 success
37 16 success
37 17 success
37 18 success
37 19 success
37 20 success
37 21 success
37 22 success
37 23 success
37 24 success
37 25 success
37 26 success
37 27 success
37 28 success
37 29 success
37 30 success
37 31 success
37 32 success
37 33 success
37 34 success
37 35 success
37 36 success
37

48 18 success
48 19 success
48 20 success
48 21 success
48 22 success
48 23 success
48 24 success
48 25 success
48 26 success
48 27 success
48 28 success
48 29 success
48 30 success
48 31 success
48 32 success
48 33 success
48 34 success
48 35 success
48 36 success
48 37 success
48 38 success
48 39 success
48 40 success
48 41 success
48 42 success
48 43 success
48 44 success
48 45 success
48 46 success
48 47 success
48 48 success
48 49 success
48 50 success
49 0 success
49 1 success
49 2 success
49 3 success
49 4 success
49 5 success
49 6 success
49 7 success
49 8 success
49 9 success
49 10 success
49 11 success
49 12 success
49 13 success
49 14 success
49 15 success
49 16 success
49 17 success
49 18 success
49 19 success
49 20 success
49 21 success
49 22 success
49 23 success
49 24 success
49 25 success
49 26 success
49 27 success
49 28 success
49 29 success
49 30 success
49 31 success
49 32 success
49 33 success
49 34 success
49 35 success
49 36 success
49 37 success
49 38 success
49

In [2]:
p = 0.8
sigma=math.sqrt((1-p)/29/29+p/300-((1-p)/29+p/30)*((1-p)/29+p/30))
mu_model = (1-p)/29+p/30
c = 0
h_List = [182,660,1155,465,46,218,270,345,1292,2235,2770,1350,195,2129,8697,2775,172,27,900,17201,1279,21780]
b_List = [1000,1000,1000,1000,1000,1000,1000,1000,5000,1000,5000,1000,1000,5000,10000,5000,1000,1000,1000,20000,5000,40000]
L_List = [19,15,8,9,9,6,9,8,16,15,6,7,2,7,5,10,11,35,10,8,7,4]

N = 22

for i in range (N): #re-scaling
    b_List[i] = b_List[i] *100 / 100.0
    h_List[i] = h_List[i] / 100.0

New_Binomial = open("New-Binomial-Simulation.txt", mode='w')
New_Poisson = open("New-Poisson-Simulation.txt", mode='w')

Structure = [[-1],[-1],[-1],[-1],[-1],[-1],[-1],[-1],[0,1],[-1],[2,3,4],[-1],[-1],[5,6,7],[8,9,10],[11,12],[-1],[-1],[-1],[13,14,15],[16,17],[18,19,20]]
S_in_supplier = [10,10,10,10,10,10,10,10,1e20,10,1e20,10,10,1e20,1e20,1e20,10,10,10,1e20,1e20,1e20]
S_out_customer = [1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,5]

S_in_List = solution[:,1].tolist()
S_out_List = solution[:,2].tolist()
B_List = solution[:,3].tolist()
W_List = solution[:,4].tolist()

Non_Start = [] #find the set of stages with upstream stages
for j in range (N):
    if (Structure[j][0] > -1):
        Non_Start.append(j)
for i in range (len(Non_Start)):
    max_S = 0
    for j in Structure[Non_Start[i]]:
        if (max_S < S_out_List[j]):
            max_S = S_out_List[j]
    S_in_List[Non_Start[i]] = max_S

print("S_in:",S_in_List)
print("S_out:",S_out_List)
print("B:",B_List)
print("W:",W_List)

#simulation-binomial
for w in range (100):
    numpy.random.seed(w+10)
    Z = numpy.random.binomial(1,p,500).tolist() #Z-29
    demand = [] #real-time demand 0 or 1
    for z in Z:
        for tempintdemand in range (28):
            demand.append(0)
        if (z == 0):
            demand.append(1)
        else:
            demand.append(0)
            demand.append(1)
    cost = numpy.zeros(N).tolist()
    backordertimes = numpy.zeros(N).tolist()
    for i in range (N):
        stock = []
        stock.append(B_List[i])
        for j in range (1, len(demand)):
            stock.append(stock[j-1])
            if ((j-S_out_List[i]) >= 0):
                stock[j] -= demand[int(j-S_out_List[i])]
            if ((j-W_List[i]-L_List[i]) >= 0):
                stock[j] += demand[int(j-W_List[i]-L_List[i])]
            if (stock[j] < 0):
                cost[i] -= stock[j]*b_List[i]
                backordertimes[i] -= stock[j]
                stock[j] = 0
            else:
                cost[i] += h_List[i]*stock[j]
    New_Binomial.write(str(w)+'\t')
    for i in range (N):
        New_Binomial.write(str(cost[i])+'\t')
    for i in range (N):
        New_Binomial.write(str(backordertimes[i])+'\t')
    New_Binomial.write('\n')
print("Simulation-Binomial Ended!")

#simulation-poisson
for w in range (100):
    numpy.random.seed(w+10)
    demand = numpy.random.poisson(mu_model,int(((1-p)*29+p*30)*500)).tolist()
    cost = numpy.zeros(N).tolist()
    backordertimes = numpy.zeros(N).tolist()
    for i in range (N):
        stock = []
        stock.append(B_List[i])
        for j in range (1, len(demand)):
            stock.append(stock[j-1])
            if ((j-S_out_List[i]) >= 0):
                stock[j] -= demand[int(j-S_out_List[i])]
            if ((j-W_List[i]-L_List[i]) >= 0):
                stock[j] += demand[int(j-W_List[i]-L_List[i])]
            if (stock[j] < 0):
                cost[i] -= stock[j]*b_List[i]
                backordertimes[i] -= stock[j]
                stock[j] = 0
            else:
                cost[i] += h_List[i]*stock[j]
    New_Poisson.write(str(w)+'\t')
    for i in range (N):
        New_Poisson.write(str(cost[i])+'\t')
    for i in range (N):
        New_Poisson.write(str(backordertimes[i])+'\t')
    New_Poisson.write('\n')
print("Simulation-Poisson Ended!")
New_Binomial.close() #every row: k,w,cost of stages 0~21, backorder times of stages 0~21
New_Poisson.close()

S_in: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 0, 10.0, 0, 10.0, 10.0, 0, 16.0, 12.0, 10.0, 10.0, 10.0, 22.0, 21.0, 1.0]
S_out: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 13.0, 0.0, 12.0, 7.0, 21.0, 22.0, 21.0, 16.0, 0.0, 1.0, 1.0, 5.0]
B: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]
W: [10.0, 14.0, 21.0, 20.0, 20.0, 23.0, 20.0, 21.0, 0.0, 14.0, 7.0, 22.0, 10.0, 0.0, 16.0, 12.0, 10.0, 10.0, 19.0, 22.0, 23.0, 1.0]
Simulation-Binomial Ended!
Simulation-Poisson Ended!
